In [14]:
import os
from pathlib import Path

# Change directory
# Modify this cell to insure that the output shows the correct path.
# Define all paths relative to the project root shown in the cell output
project_root = "/home/sscf/freqtrade"
i=0
try:
    os.chdirdir(project_root)
    assert Path('LICENSE').is_file()
except:
    while i<4 and (not Path('LICENSE').is_file()):
        os.chdir(Path(Path.cwd(), '../'))
        i+=1
    project_root = Path.cwd()
print(Path.cwd())

/freqtrade


In [15]:
import numpy as np
import pandas as pd

In [29]:
from freqtrade.configuration import Configuration


# Customize these according to your needs.

# Initialize empty configuration object
# Optionally (recommended), use existing configuration file
config = Configuration.from_files(["user_data/config.json"])

# Define some constants
config["timeframe"] = "5m"
# Name of the strategy class
config["strategy"] = "RandomEntry"
# Location of the data
data_location = config["datadir"]
# Pair to analyze - Only use one pair here
pair = "MELANIA/USDT:USDT"

2025-06-08 13:20:29,869 - freqtrade.configuration.load_config - INFO - Using config: user_data/config.json ...

2025-06-08 13:20:30,096 - freqtrade.loggers - INFO - Enabling colorized output.

2025-06-08 13:20:30,100 - root - INFO - Logfile configured

2025-06-08 13:20:30,118 - freqtrade.loggers - INFO - Verbosity set to 0

2025-06-08 13:20:30,476 - freqtrade.configuration.configuration - INFO - Using user-data directory: /freqtrade/user_data ...

2025-06-08 13:20:30,480 - freqtrade.configuration.configuration - INFO - Using data directory: /freqtrade/user_data/data/binance ...

2025-06-08 13:20:30,482 - freqtrade.exchange.check_exchange - INFO - Checking exchange...

2025-06-08 13:20:30,524 - freqtrade.exchange.check_exchange - INFO - Exchange "binance" is officially supported by the Freqtrade development team.

2025-06-08 13:20:30,527 - freqtrade.configuration.configuration - INFO - Using pairlist from configuration.

In [23]:
# Encuentra archivos que contengan "MELANIA"
import os
for root, dirs, files in os.walk(data_location):
    for file in files:
        if "MELANIA" in file.upper():
            print(f"Encontrado: {os.path.join(root, file)}")

Encontrado: /freqtrade/user_data/data/binance/futures/MELANIA_USDT_USDT-5m-futures.feather
Encontrado: /freqtrade/user_data/data/binance/futures/MELANIA_USDT_USDT-8h-funding_rate.feather
Encontrado: /freqtrade/user_data/data/binance/futures/MELANIA_USDT_USDT-8h-mark.feather


In [24]:
# Load data using values set above
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType


data_location = Path(config['user_data_dir'], 'data', 'binance')

candles = load_pair_history(
    datadir=data_location,
    timeframe=config["timeframe"],
    pair=pair,
    data_format="feather",  # Make sure to update this to your data
    candle_type=CandleType.FUTURES,
)

# Confirm success
print(f"Loaded {len(candles)} rows of data for {pair} from {data_location}")
candles.head()

Loaded 37902 rows of data for MELANIA/USDT:USDT from /freqtrade/user_data/data/binance


,date,open,high,low,close,volume
0,2025-01-20 09:30:00+00:00,11.000,11.377,9.570,10.671,3170927.05
1,2025-01-20 09:35:00+00:00,10.680,11.126,10.248,10.877,1890644.05
2,2025-01-20 09:40:00+00:00,10.882,11.310,10.802,11.198,1415396.91
3,2025-01-20 09:45:00+00:00,11.198,11.481,10.800,11.053,1628204.05
4,2025-01-20 09:50:00+00:00,11.053,11.137,10.301,10.502,1799539.90


In [30]:
# Load strategy using values set above
from freqtrade.data.dataprovider import DataProvider
from freqtrade.resolvers import StrategyResolver


strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)
strategy.ft_bot_start()

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {"pair": pair})
df.tail()

2025-06-08 13:20:33,340 - freqtrade.resolvers.iresolver - INFO - Using resolved strategy RandomEntry from '/freqtrade/user_data/strategies/RandomEntry.py'...

2025-06-08 13:20:33,346 - freqtrade.strategy.hyper - INFO - Found no parameter file.

2025-06-08 13:20:33,350 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'timeframe' with value in config file: 5m.

2025-06-08 13:20:33,354 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'process_only_new_candles' with value in config file: True.

2025-06-08 13:20:33,357 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'stake_currency' with value in config file: USDT.

...

2025-06-08 13:20:33,524 - freqtrade.strategy.hyper - INFO - No params for buy found, using default values.

2025-06-08 13:20:33,529 - freqtrade.strategy.hyper - INFO - Strategy Parameter(default): long_probability = 0.5

2025-06-08 13:20:33,533 - freqtrade.strategy.hyper - INFO - No params for sell found, using default values.

2025-06-08 13:20:33,536 - freqtrade.strategy.hyper - INFO - No params for protection found, using default values.

,date,open,high,low,close,volume,random_value,random_long,random_short,enter_tag,enter_long,enter_short,exit_tag
37897,2025-05-31 23:35:00+00:00,0.3046,0.3053,0.3046,0.3049,55120.35,0.013859,True,False,,1,0,
37898,2025-05-31 23:40:00+00:00,0.3050,0.3050,0.3045,0.3045,51640.07,0.502506,False,True,,0,1,
37899,2025-05-31 23:45:00+00:00,0.3047,0.3051,0.3045,0.3048,32958.30,0.534789,False,True,,0,1,
37900,2025-05-31 23:50:00+00:00,0.3048,0.3056,0.3046,0.3049,36045.36,0.931097,False,True,,0,1,
37901,2025-05-31 23:55:00+00:00,0.3050,0.3054,0.3046,0.3051,35735.12,0.850837,False,True,,0,1,


In [34]:
# Report results
print(f"Generated {df['enter_long'].sum()} entry signals")
print(f"Generated {df['enter_short'].sum()} entry signals")
data = df.set_index("date", drop=False)
data.tail()

Generated 18809 entry signals
Generated 19093 entry signals


,date,open,high,low,close,volume,random_value,random_long,random_short,enter_tag,enter_long,enter_short,exit_tag
date,,,,,,,,,,,,,
2025-05-31 23:35:00+00:00,2025-05-31 23:35:00+00:00,0.3046,0.3053,0.3046,0.3049,55120.35,0.013859,True,False,,1,0,
2025-05-31 23:40:00+00:00,2025-05-31 23:40:00+00:00,0.3050,0.3050,0.3045,0.3045,51640.07,0.502506,False,True,,0,1,
2025-05-31 23:45:00+00:00,2025-05-31 23:45:00+00:00,0.3047,0.3051,0.3045,0.3048,32958.30,0.534789,False,True,,0,1,
2025-05-31 23:50:00+00:00,2025-05-31 23:50:00+00:00,0.3048,0.3056,0.3046,0.3049,36045.36,0.931097,False,True,,0,1,
2025-05-31 23:55:00+00:00,2025-05-31 23:55:00+00:00,0.3050,0.3054,0.3046,0.3051,35735.12,0.850837,False,True,,0,1,
